In [1]:
library(parallel)
library(stringr)

In [2]:
vcf_dir = '/nfs/lab/projects/pbmc_snATAC/pipeline/snp_array/final_vcfs/'

In [3]:
homedir = '/nfs/lab/projects/pbmc_snATAC/analysis_v2/rasqual/'

In [4]:
samples = paste0("PBMC", c(1:6, 8:10,12:15))

### FIlter original vcf files for just  snps in peaks, samples of interest, imputation quality and heterozygous genotype and remove indels

In [28]:
filter_vcf = function(cell) {
        
    for (n in 1:22) {

    vcf    = paste0(vcf_dir, "/chr",n ,'.PBMC.topmed.hg19.final.vcf.gz')
    outdir = paste(homedir, reso, cell, "vcfs_peaksonly", sep="/")    
    system(paste("mkdir -p ",outdir  ) )   
    outvcf = paste0(outdir, "/chr",n ,'.filt.vcf')
    bed    = paste0(homedir, reso, "/",cell, "/" , cell, '.filtered.bed')
        
    system(paste('bcftools view', vcf ,
    '-R' , bed, 
    "-i \'INFO/R2[0]>0.7\' | bcftools view -g het --types snps -o" , outvcf))
}
    }

In [5]:
reso      = "bulk"
celltypes = "bulk"
cell= 'bulk'

In [6]:
ids = c(1:6, 8:10,12:15)

In [33]:
mclapply(celltypes, function(x) filter_vcf(x) , mc.cores = 32)

[[1]]
NULL

In [7]:
# for Bulk: use the file created by removing duplicate reads on a "bulk-basis"(as opposed to bc)
new_vcf_dir = paste0(homedir, reso,"/" ,cell, "/vcfs_peaksonly" )    
sampdir     = "/nfs/lab/projects/pbmc_snATAC/analysis_v2/wasp_peaks/bulk"
bam         = paste0(sampdir, "/PBMC", ids, '/rmdup/PBMC', ids, '.keep.merged.rmdup.bam' )


sorted_bam= paste0(sampdir, "/PBMC", ids, '/rmdup/PBMC', ids, '.keep.merged.sorted.rmdup.bam' )

writeLines(sorted_bam, paste(new_vcf_dir, 'bam.list.txt', sep="/"))

In [19]:
# ## sort and Index the bam files
sort_and_index = function(b){
    system(paste0("cd ", sampdir,"/PBMC", ids[b], "/rmdup"))
    system(paste('samtools', 'sort', '-m', '2G', '-@', '3', '-o', sorted_bam[b], bam[b])) 
    system(paste("samtools index", sorted_bam[b]))
}

In [20]:
mclapply(1:13, function (x) sort_and_index(x),mc.cores = 48)

[[1]]
[1] 0

[[2]]
[1] 0

[[3]]
[1] 0

[[4]]
[1] 0

[[5]]
[1] 0

[[6]]
[1] 0

[[7]]
[1] 0

[[8]]
[1] 0

[[9]]
[1] 0

[[10]]
[1] 0

[[11]]
[1] 0

[[12]]
[1] 0

[[13]]
[1] 0

In [12]:
rasqual_ase = function (cell, reso, chr){
setwd(paste0(homedir, reso,"/" ,cell, "/vcfs_peaksonly" )  )
outfile = paste0( "chr",chr, '.ase.filtered.vcf.gz')   
system("rm full.ase*")   ## remove older files 
system(paste0('bgzip chr', chr, '.filt.vcf -f' )) 
system(paste0('tabix chr', chr, '.filt.vcf.gz -f' ))     
system(paste0('sh $RASQUALDIR/src/ASVCF/createASVCF.sh paired_end bam.list.txt chr',chr,
              '.filt.vcf.gz chr', chr, '.ase.vcf.gz atac'))
system(paste0('/home/pbenaglio/bin/bcftools view chr', 
              chr, '.ase.vcf.gz -i \'COUNT(GT=\"het\")>1\' -Oz -o ', outfile))
system(paste('tabix', outfile, '-f'))
    }

In [13]:
mclapply(1:22, function (x) rasqual_ase(cell='bulk', reso=reso,chr=x ),mc.cores = 40)

[[1]]
[1] 0

[[2]]
[1] 0

[[3]]
[1] 0

[[4]]
[1] 0

[[5]]
[1] 0

[[6]]
[1] 0

[[7]]
[1] 0

[[8]]
[1] 0

[[9]]
[1] 0

[[10]]
[1] 0

[[11]]
[1] 0

[[12]]
[1] 0

[[13]]
[1] 0

[[14]]
[1] 0

[[15]]
[1] 0

[[16]]
[1] 0

[[17]]
[1] 0

[[18]]
[1] 0

[[19]]
[1] 0

[[20]]
[1] 0

[[21]]
[1] 0

[[22]]
[1] 0

In [17]:
chromosomeASE= paste(paste0( "chr",1:22, '.ase.filtered.vcf.gz') , collapse=" ")

In [18]:
setwd(paste(homedir, reso, cell, "vcfs_peaksonly", sep="/"))
system(paste("bcftools concat", chromosomeASE, "-Oz -o full.ase.filtered.vcf.gz"))
system('tabix full.ase.filtered.vcf.gz -f')
    

In [19]:
for (cell in celltypes){
setwd(paste0(homedir,reso ,"/", cell, "/vcfs_peaksonly"))
system( "zgrep \'#\' full.ase.filtered.vcf.gz > header") 
test = read.table('full.ase.filtered.vcf.gz')
for(n in 1:2){
for( i in 10:22){
tabs  = str_split_fixed(test[,i], "\\:", 5)
ac    = tabs[tabs[,1]=="0|1" | tabs[,1]=="1|0", 5]
acs   = str_split(ac,",")
acsp  = t(sapply(acs, sample))
acsp  = apply(acsp, 1, str_flatten, collapse=",")
tabs2 = tabs
tabs2[tabs2[,1]=="0|1" | tabs2[,1]=="1|0", 5] <- acsp

tt    =  apply(tabs2, 1, str_flatten, collapse=":")
   
    if (i ==10){
        TT = tt  }else{
        TT = cbind(TT,tt)}
    }

europ = c(2,3,4,5,7,8,9,10,11,12)
shuff = sample(europ)

if(n==1) {
    perm =  c(6, shuff[1:4], 13, shuff[5:10],1)
} else if(n==2) {
    perm =  c(13, shuff[1:4], 1, shuff[5:10],6)}
    
TT = TT[,perm]
testp = cbind(test[,1:9], TT)

outfile = paste0("full.ase.filtered.allelic.perm",n)
setwd(paste0(homedir,reso ,"/", cell, "/vcfs_peaksonly"))
write.table(testp, outfile, col.names=F, row.names=F, quote=F, sep="\t" )
system(paste0('cat header ', outfile, ' > ', outfile, '.vcf'))
system(paste0('bgzip ', outfile, '.vcf -f'))
system(paste0('tabix ', outfile, '.vcf.gz -f'))
system(paste("rm", outfile))    
    }
    }

In [32]:
head(tabs)

0|0,0,"1,0,0","0,0","0,0"
1|1,1.932,"0.001,0.066,0.933","0.961,0.971","0,0"
0|0,0,"1,0,0","0,0","0,0"
0|0,0,"1,0,0","0,0","0,0"
0|0,0,"1,0,0","0,0","0,0"
0|0,0.001,"0.999,0.001,0","0,0.001","0,0"
